<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Image2Matrix and Matrix2Image Functions in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> Image2Matrix function converts an image to a matrix. It converts JPEG or PNG images to matrices with payload values being the pixel values. Matrix2Image converts a matrix to an image. The conversion produces an image using color maps. The color image produced by Matrix2Image is limited to 8-bit color depth.<br>
    In this notebook we will see how we can use the Image2Matrix and Matrix2Image functions available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>We need to upgarde teradataml to 20.0.0.3 version or higher to functions to work properly.</p>


In [ ]:
%%capture
!pip install --upgrade teradataml

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *
from PIL import Image
from IPython.display import display
import io

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Image2MatrixandMatrix2Image_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We will first load the image as blob data in a table and then use the functions. First step is to create the table.</p>

In [ ]:
# Drop the image table if it is present.
try:
    db_drop_table('image_table')
except:
    pass

# Create a table to store the image data.
execute_sql('CREATE TABLE image_table(id INTEGER, image BLOB);')

<p style = 'font-size:16px;font-family:Arial'>Now we can load this image into the table.</p>

In [ ]:
import teradataml
file_dir = os.path.join(os.path.dirname(teradataml.__file__), "data")

# Open the JPEG image
image = Image.open(os.path.join(file_dir,'peppers.png'))  # Replace with actual file path

# Display the image
display(image)

In [ ]:
# Load the image data into the fileContent variable.
with open(os.path.join(file_dir,'peppers.png'), mode='rb') as file:
    fileContent = file.read()

# Insert the image data into the table.
sql = 'INSERT INTO image_table VALUES(?, ?);'
parameters = (1, fileContent)
execute_sql(sql, parameters)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
# Create a DataFrame for the image table.
imageTable = DataFrame('image_table')
imageTable

<p style = 'font-size:16px;font-family:Arial'>Now we have loaded the image we will convert the image to matrix using the Image2Matrix function. Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(Image2Matrix)

In [ ]:
# Example: Convert the image to matrix with gray values.
image2matrix_gray = Image2Matrix(data=imageTable.select(['id', 'image']), 
                            output='GRAY')

# Print the result DataFrame.
image2matrix_gray.result

In [ ]:
image2matrix_gray.result.shape

In [ ]:
# Example: Convert the image to matrix with rgb values.
image2matrix_rgb = Image2Matrix(data=imageTable.select(['id', 'image']), 
                            output='RGB')

# Print the result DataFrame.
image2matrix_rgb.result

<p style = 'font-size:16px;font-family:Arial'>As we have now converted our image as a matrix we can create filters or any image manipulation using matrix calculations. After that we can convert the matrix to image again using Matrix2Image function. Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(Matrix2Image)

<p style = 'font-size:16px;font-family:Arial'>As a first example let us first invert the grayscale image</p>

In [ ]:
#creating inverse dataframe
df = image2matrix_gray.result
invert_df= df.assign(drop_columns = True,
                  id = df.id,
                  Y= df.Y,
                  X=df.X,
                  GRAY=255-df.GRAY
                   )

# Create teradataml TDMatrix object.
data_matrix_df = TDMatrix(data=invert_df,
                          id="id",
                          row_index="Y",
                          column_index="X",
                          row_index_style="SEQUENCE",
                          column_index_style="SEQUENCE",
                          payload_field="GRAY",
                          payload_content="REAL"
                          )

# Example 1: Generate Gray Scale Image Output with Fixed Range.
inverse_image = Matrix2Image(data=data_matrix_df,
                       range=[0,255])

# Print the result DataFrame.
inverse_image.result

In [ ]:
copy_to_sql(inverse_image.result,table_name="image_inverse",if_exists="replace")

<p style = 'font-size:16px;font-family:Arial'>The output image looks like this.</p>

In [ ]:
dis_df = DataFrame(in_schema("demo_user","image_inverse"))
# Convert the 'image_blob_column' to bytes
blob_data = dis_df.to_pandas()['IMAGE'][0]  # Get first image
    
# Convert BLOB to Image
image = Image.open(io.BytesIO(blob_data))

# Display Image
display(image)

<p style = 'font-size:16px;font-family:Arial'>As a second example we will flip the rgb image on y-axis.</p>

In [ ]:
# Create teradataml TDMatrix object.
data_matrix_df2 = TDMatrix(data=image2matrix_rgb.result,
                          id="id",
                          row_index="Y",
                          column_index="X",
                          row_index_style="SEQUENCE",
                          column_index_style="SEQUENCE",
                          payload_field=["RED", "BLUE", "GREEN"],
                          payload_content="MULTIVAR_REAL"
                          )

# Example 4: Generate RGB Image Output with All Channels Range Fixed.
flip_image = Matrix2Image(data=data_matrix_df2,
                          red=[0,255],
                          green=[0,255],
                          blue=[0,255],
                          flip_y=True)

# Print the result DataFrame.
flip_image.result

In [ ]:
copy_to_sql(flip_image.result,table_name="image_flip",if_exists="replace")

<p style = 'font-size:16px;font-family:Arial'>Let us view the output image.</p>

In [ ]:
dis_df_flip = DataFrame(in_schema("demo_user","image_flip"))
# Convert the 'image_blob_column' to bytes
blob_data = dis_df_flip.to_pandas()['IMAGE'][0]  # Get first image
    
# Convert BLOB to Image
image = Image.open(io.BytesIO(blob_data))

# Display Image
display(image)

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:##00233C'><b>Work Tables</b></p>

In [ ]:
db_drop_table('image_table')

In [ ]:
db_drop_table('image_flip')

In [ ]:
db_drop_table('image_inverse')

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Image2Matrix function reference: <a href = 'https://docs.teradata.com/search/all?query=Image2Matrix&content-lang=en-US'>here</a></li>
    <li>Matrix2Image function reference: <a href = 'https://docs.teradata.com/search/all?query=Matrix2Image&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>